#Objective

#Notebook Initialization

##Import Packages

In [0]:
from pyspark.sql.functions import col,isnan,when,count,lit, to_date,lpad,date_format,rpad,regexp_replace,concat,to_utc_timestamp,to_timestamp, countDistinct,unix_timestamp, row_number, when
from pyspark.sql.types import IntegerType,BooleanType,DateType,StringType,TimestampType
from pyspark.sql import DataFrameNaFunctions
from pyspark import StorageLevel
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pytz import timezone
import datetime
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler, PCA, VectorSlicer, Imputer
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as f

##Cloud Storage Parameters

In [0]:
blob_container = "tm30container" # The name of your container created in https://portal.azure.com
storage_account = "w261tm30" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261tm30" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "tm30key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

test_pq = spark.read.parquet(f"{blob_url}/2022-03-24_data_chkpt_PQ_full")

##Define Functions

In [0]:
def data_pull(df, time_window = 'full', date_col='FLIGHT_UTC_DATE'):
    """Pull processed dataset"""
    if time_window == '2019':
        df = df.filter(f.year(col(date_col)) == 2019)
    elif time_window == '2018':
        df = df.filter(f.year(col(date_col)) == 2018)
    elif time_window == '2017':
        df = df.filter(f.year(col(date_col)) == 2017)
    elif time_window == '2016':
        df = df.filter(f.year(col(date_col)) == 2016) 
    
    #The commands below are for 2015 data
    elif time_window == '6m':
        df = df.filter(col(date_col) < "2015-07-01T00:00:00.000")  
    elif time_window == '3m':
        df = df.filter(col(date_col) < "2015-04-01T00:00:00.000")
        #comment this out if it takes too long
    
    print(f'{df.count():,} total records imported for the {time_window} dataset')
    return df

In [0]:
def pre_pipeline(index_cols, cont_cols, cat_cols, pred_cols):
    ''' This function creates a pre-processed pipeline to be used to prepare for crossfold validation and model training
    '''
    pre_pipeline = None
    
    #Convert string to index
    indexer = StringIndexer(inputCols=cat_cols, outputCols=[c+"_idx" for c in cat_cols]).setHandleInvalid("keep")

    #Convert categorical columns to index
    encoder = OneHotEncoder(inputCols=[c+"_idx" for c in cat_cols], outputCols= [c+"_OHE" for c in cat_cols])
    
    #Vector assembler for categorical
    assembler_cat = VectorAssembler(inputCols= [x+"_OHE" for x in cat_cols], outputCol="cat_features")
        
    assembler_lab = StringIndexer(inputCol='DEP_DEL15', outputCol="label")
        
    pre_pipeline = Pipeline(stages=[indexer, encoder, assembler_cat, assembler_lab])
    
    return pre_pipeline

In [0]:
def scaled_pipeline(model, param_grid):
    ''' This function creates a scaled processed and scaled pipeline to be used to train models.
        Parameters:
            model:    lr = Logistic Regression;
                      rf = Random Forest 
                      dt = Decision Trees
        Returns: a pipeline model
    '''
    pipeline_model = None
    
    #Ensure continuous variables have values
    imputer = Imputer(inputCols=cont_cols, outputCols=cont_cols)
    
    #Assemble cont variables
    assembler_num = VectorAssembler(inputCols=cont_cols, outputCol="scale_nums")
    
    #Scale the values
    scaler = StandardScaler(inputCol="scale_nums", outputCol="scaledFeatures", withStd=True, withMean=True)

    #Vector assembler combined
    assembler = VectorAssembler(inputCols=["scaledFeatures", "cat_features"], outputCol="features")
    
    #Models for the pipeline
    if model == 'lr':
        max_iter, reg_param, ela = param_grid.values()
        
        class_model = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter = max_iter, regParam = reg_param, elasticNetParam = ela)
        
    elif model == 'rf':
        num_trees, depth, criterion, features = param_grid.values()
        
        class_model = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', numTrees = num_trees, maxDepth = depth, impurity = criterion, featureSubsetStrategy = features, maxMemoryInMB = 512, minInfoGain =  1e-5, cacheNodeIds = True)
        
    elif model == 'dt':
        md, mb, info = param_grid.values()
    
        class_model = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = md, maxBins = mb, impurity = info, maxMemoryInMB = 512, minInfoGain =  1e-5, cacheNodeIds = True)
        
    pipeline_model = Pipeline(stages=[imputer, assembler_num, scaler, assembler, class_model])
    
    return pipeline_model

In [0]:
def custom_CV(df_rank, pre_pipeline, class_model, sample, kfolds):

    #Create evaluation metric lists
    f_5_score_list_CV_average = []

    #Logistic Model Lists
    maxiteration = []
    regulationparameter = []
    elasticnet = []
    
    #Decision Tree Model Lists
    maxdepth = []
    maxbins = []
    infotype = []
    
    #Random Forest Model Lists
    numtrees = []
    depthtree = []
    criterion_type = []
    feature_subset = []
    
    #Preprocess Pipleine
    fit_df = pre_pipeline.fit(df_rank)
    transform_df = fit_df.transform(df_rank).persist(StorageLevel.MEMORY_AND_DISK)
    
    if class_model == 'lr':
        maxiter = [10, 100]
        regulation_param = [0.01, 0.5, 2.0]
        elastic = [0.0, 0.5, 1.0]
        
        for max_iter in maxiter:
            for reg_param in regulation_param:
                for ela in elastic:
                    
                    # Create Time Splits
                    splits = 1.0/(kfolds + 1)
                    cutoff = splits
                    
                    f_5_score_list = []
                    param_grid = {'maxIter':max_iter, 'regParam': reg_param, 'elasticNetParam':ela}
                    
                    scaled_pipelines = scaled_pipeline('lr', param_grid)
                    
                    for split in range(kfolds):
                        
                        train_df = transform_df.where(f"rank <= {cutoff}").cache()
                        test_df = transform_df.where(f"rank > {cutoff} and rank <= {cutoff+splits}").cache()
                        cutoff += splits
                        
                        if sample == 'down':
                            train_df = downsample(train_df)
                            
                        #Generate model 
                        model = scaled_pipelines.fit(train_df)
                        predict = model.transform(test_df)
                        
#                       Calculate evaluation metrics
                        evaluatorf_5 = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', metricLabel=1, beta=0.5)
                        f_5 = evaluatorf_5.evaluate(predict)
                        f_5_score_list.append(f_5)
                    
                    f_score_avg = np.mean(f_5_score_list)
                    
                    f_5_score_list_CV_average.append(f_score_avg)
                    maxiteration.append(max_iter)
                    regulationparameter.append(reg_param)
                    elasticnet.append(ela)

                        
                    print('F Score: {:3f}\nParam Grid: {}'.format(f_score_avg, param_grid.items()))
                    
                    
        Eval_df = pd.DataFrame()
        Eval_df['F_0.5 Score'] = f_5_score_list_CV_average
        Eval_df['Max Iterations'] = maxiteration
        Eval_df['Regulation Parameter'] = regulationparameter
        Eval_df['Elastic Net'] = elasticnet
        
    elif class_model == 'dt':
        max_depth = [5, 10, 20]
        max_bins = [6168]
        info_type = ['gini']
        
        for md in max_depth:
            for mb in max_bins:
                for it in info_type:
                    
                    # Create Time Splits
                    splits = 1.0/(kfolds + 1)
                    cutoff = splits
                    
                    f_5_score_list = []
                    param_grid = {'max_depth':md, 'max_bins': mb, 'info_type':it}
                    
                    scaled_pipelines = scaled_pipeline('dt', param_grid)
                    
                    for split in range(kfolds):
                        
                        train_df = transform_df.where(f"rank <= {cutoff}").cache()
                        test_df = transform_df.where(f"rank > {cutoff} and rank <= {cutoff+splits}").cache()
                        cutoff += splits
                        
                        if sample == 'down':
                            train_df = downsample(train_df)
                            
                        #Generate model 
                        model = scaled_pipelines.fit(train_df)
                        predict = model.transform(test_df)
                        
#                       Calculate evaluation metrics
                        evaluatorf_5 = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', metricLabel=1, beta=0.5)
                        f_5 = evaluatorf_5.evaluate(predict)
                        f_5_score_list.append(f_5)
                    
                    f_score_avg = np.mean(f_5_score_list)
                    
                    f_5_score_list_CV_average.append(f_score_avg)
                    maxdepth.append(md)
                    maxbins.append(mb)
                    infotype.append(it)
                        
                    print('F Score: {:3f}\nParam Grid: {}'.format(f_score_avg, param_grid.items()))
                    
                    
        Eval_df = pd.DataFrame()
        Eval_df['F_0.5 Score'] = f_5_score_list_CV_average
        Eval_df['Max Depth'] = maxdepth
        Eval_df['Max Bins'] = maxbins
        Eval_df['Info Type'] = infotype

    elif class_model == 'rf':
        num_trees = [10, 20, 50]
        depth = [3, 5, 10]
        criterion = ['gini']
        feature_subsetstrat = ['sqrt', 'all']
        
        for nt in num_trees:
            for d in depth:
                for crit in criterion:
                    for feature in feature_subsetstrat:
                    
                        # Create Time Splits
                        splits = 1.0/(kfolds + 1)
                        cutoff = splits

                        f_5_score_list = []
                        param_grid = {'num_trees':nt, 'depth': d, 'criterion':crit, 'substrategy':feature}

                        scaled_pipelines = scaled_pipeline('rf', param_grid)

                        for split in range(kfolds):

                            train_df = transform_df.where(f"rank <= {cutoff}").cache()
                            test_df = transform_df.where(f"rank > {cutoff} and rank <= {cutoff+splits}").cache()
                            cutoff += splits

                            if sample == 'down':
                                train_df = downsample(train_df)

                            #Generate model 
                            model = scaled_pipelines.fit(train_df)
                            predict = model.transform(test_df)

    #                       Calculate evaluation metrics
                            evaluatorf_5 = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', metricLabel=1, beta=0.5)
                            f_5 = evaluatorf_5.evaluate(predict)
                            f_5_score_list.append(f_5)

                        f_score_avg = np.mean(f_5_score_list)

                        f_5_score_list_CV_average.append(f_score_avg)
                        numtrees.append(nt)
                        depthtree.append(d)
                        criterion_type.append(crit)
                        feature_subset.append(feature)

                        print('F Score: {:3f}\nParam Grid: {}'.format(f_score_avg, param_grid.items()))
                    
                    
        Eval_df = pd.DataFrame()
        Eval_df['F_0.5 Score'] = f_5_score_list_CV_average
        Eval_df['Tree Number'] = numtrees
        Eval_df['Depth'] = depthtree
        Eval_df['Info Type'] = criterion_type
        Eval_df['Feature Substrategy'] = feature_subset
        
    transform_df.unpersist()
    
    return Eval_df

In [0]:
def threshold_CV(df_rank, pre_pipeline, class_model, sample, kfolds):

    #Create evaluation metric lists
    f_5_score_list_CV_average = []
    threshold_list = []
    
    #Logistic Model Lists
    threshold = [.2, .3, .4, .6, .7, .8, .9]
    
    #Preprocess Pipleine
    fit_df = pre_pipeline.fit(df_rank)
    transform_df = fit_df.transform(df_rank).persist(StorageLevel.MEMORY_AND_DISK)
    
    if class_model == 'lr':
        
        for thresh in threshold:
                    
                    # Create Time Splits
            splits = 1.0/(kfolds + 1)
            cutoff = splits
                    
            f_5_score_list = []
            param_grid = {'maxIter':10, 'regParam': 0, 'elasticNetParam':1, 'threshold':thresh}
                    
            scaled_pipelines = scaled_pipeline('lr', param_grid)
                    
            for split in range(kfolds):
                        
                train_df = transform_df.where(f"rank <= {cutoff}").cache()
                test_df = transform_df.where(f"rank > {cutoff} and rank <= {cutoff+splits}").cache()
                cutoff += splits
                        
                if sample == 'down':
                    train_df = downsample(train_df)
                            
                        #Generate model 
                model = scaled_pipelines.fit(train_df)
                predict = model.transform(test_df)
                        
#                       Calculate evaluation metrics
                evaluatorf_5 = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', metricLabel=1, beta=0.5)
                f_5 = evaluatorf_5.evaluate(predict)
                f_5_score_list.append(f_5)
                    
            f_score_avg = np.mean(f_5_score_list)
                    
            f_5_score_list_CV_average.append(f_score_avg)
            threshold_list.append(thresh)
                        
            print('F Score: {:3f}\nParam Grid: {}'.format(f_score_avg, param_grid.items()))
                    
                    
        Eval_df = pd.DataFrame()
        Eval_df['F_0.5 Score'] = f_5_score_list_CV_average
        Eval_df['Threshhold'] = threshold_list
        
    transform_df.unpersist()
    
    return Eval_df

In [0]:
def downsample(train_df):
    '''Downsample minority class to balance classes. 
    Only works if delay count is less than on_time count (should be the case...)'''
    
    delay_count = train_df.filter(col("label") == 1).count()
    on_time_count = train_df.filter(col("label") == 0).count()
    
    total_count = delay_count + on_time_count
    delay_pct = delay_count / on_time_count
    
    train_delay = train_df.filter(col('label') == 1)
    train_on_time = train_df.filter(col('label') == 0).sample(withReplacement=False, fraction = delay_pct, seed= 2022)
    train_downsampled = train_delay.union(train_on_time)
    
    return train_downsampled

#Pipeline Initialization

In [0]:
#Last minute data curation...
test_pq = test_pq.na.replace('', None, 'wnd_type')\
            .na.replace('', None, 'ga1_cld')\
            .na.replace('', None, 'ga1_cov')\
            .withColumn('wnd_dir_angle',col('wnd_dir_angle').cast(IntegerType()))\
            .withColumn('ka1_temp', when(f.isnull('ka1_temp'), '0').when(f.col('ka1_temp') < 0, -1).otherwise('1'))\
            .withColumn('FLIGHT_ROUTE', concat(col('ORIGIN'),lit("-"),col('DEST')))

df_2015_2018 = test_pq.filter(col('FLIGHT_UTC_DATE') < "2019-01-01T00:00:00.000")

df_6m = data_pull(test_pq, time_window='6m', date_col='FLIGHT_UTC_DATE')

df_2019 = data_pull(test_pq, time_window='2019', date_col='FLIGHT_UTC_DATE')

df_small_test = test_pq.filter(col('FLIGHT_UTC_DATE') < "2015-02-01T00:00:00.000")

3,365,522 total records imported for the 6m dataset
8,539,842 total records imported for the 2019 dataset

In [0]:
(df_2015_2018.count(), len(df_2015_2018.columns))

Out[53]: (27959979, 100)

#Create Baseline Models

In [0]:
#select columns
index_cols = ['UNIQUE_ID','FLIGHT_UTC_DATE', 'rank']
cat_cols = ['TIME_OF_DAY', 'MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'wnd_type', 'cig_ceil_is_qual', 'tmp_air_is_qual',  'slp_prs_is_qual', 'ga1_cov','ga1_cld', 'ga1_bs_ht_is_qual', 'wnd_spd_is_qual', 'ga1_cld_qual', 'dew_pnt_is_qual', 'ga1_cov_is_qual', 'aa1_is_qual', 'vis_dist_is_qual', 'ka1_temp', 'FLIGHT_ROUTE']
cont_cols = ['ELEVATION', 'wnd_dir_angle', 'wnd_spd_rate', 'cig_ceil_ht', 'vis_dist', 'tmp_air', 'dew_pnt_tmp','slp_prs', 'aa1_prd_quant_hr', 'aa1_dp', 'ga1_bs_ht']
pred_cols = ['DEP_DEL15']

#Modify to switch from test to scaled up model
df_model = df_2015_2018

#Add rank to allow forx custom crossvalidation and windowing
train_test_window = df_model.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("FLIGHT_UTC_DATE")))

#Initialize Pre-processing Pipeline
pre_pipe = pre_pipeline(index_cols, cont_cols, cat_cols, pred_cols)

##Logistic Regression

In [0]:
Eval_lr = custom_CV(train_test_window, pre_pipe, 'lr', 'down', 5)
Eval_lr

F Score: 0.323845
Param Grid: dict_items([('maxIter', 10), ('regParam', 0.01), ('elasticNetParam', 0.0)])
F Score: 0.311193
Param Grid: dict_items([('maxIter', 10), ('regParam', 0.01), ('elasticNetParam', 0.5)])
F Score: 0.306506
Param Grid: dict_items([('maxIter', 10), ('regParam', 0.01), ('elasticNetParam', 1.0)])
F Score: 0.320416
Param Grid: dict_items([('maxIter', 10), ('regParam', 0.5), ('elasticNetParam', 0.0)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 10), ('regParam', 0.5), ('elasticNetParam', 0.5)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 10), ('regParam', 0.5), ('elasticNetParam', 1.0)])
F Score: 0.315576
Param Grid: dict_items([('maxIter', 10), ('regParam', 2.0), ('elasticNetParam', 0.0)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 10), ('regParam', 2.0), ('elasticNetParam', 0.5)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 10), ('regParam', 2.0), ('elasticNetParam', 1.0)])
F Score: 0.326902
Param Grid: dict_items([('maxIter', 100), ('regParam', 0.01), ('elasticNetParam', 0.0)])
F Score: 0.312312
Param Grid: dict_items([('maxIter', 100), ('regParam', 0.01), ('elasticNetParam', 0.5)])
F Score: 0.306830
Param Grid: dict_items([('maxIter', 100), ('regParam', 0.01), ('elasticNetParam', 1.0)])
F Score: 0.320402
Param Grid: dict_items([('maxIter', 100), ('regParam', 0.5), ('elasticNetParam', 0.0)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 100), ('regParam', 0.5), ('elasticNetParam', 0.5)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 100), ('regParam', 0.5), ('elasticNetParam', 1.0)])
F Score: 0.315590
Param Grid: dict_items([('maxIter', 100), ('regParam', 2.0), ('elasticNetParam', 0.0)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 100), ('regParam', 2.0), ('elasticNetParam', 0.5)])
F Score: 0.094082
Param Grid: dict_items([('maxIter', 100), ('regParam', 2.0), ('elasticNetParam', 1.0)])
Out[10]:

,F_0.5 Score,Max Iterations,Regulation Parameter,Elastic Net
0,0.323845,10,0.01,0.0
1,0.311193,10,0.01,0.5
2,0.306506,10,0.01,1.0
3,0.320416,10,0.50,0.0
4,0.094082,10,0.50,0.5
5,0.094082,10,0.50,1.0
6,0.315576,10,2.00,0.0
7,0.094082,10,2.00,0.5
8,0.094082,10,2.00,1.0
9,0.326902,100,0.01,0.0


##Decision Tree

In [0]:
Eval_dt = custom_CV(train_test_window, pre_pipe, 'dt', 'down', 5)
Eval_dt

F Score: 0.305253
Param Grid: dict_items([('max_depth', 5), ('max_bins', 6168), ('info_type', 'gini')])
F Score: 0.324010
Param Grid: dict_items([('max_depth', 10), ('max_bins', 6168), ('info_type', 'gini')])
F Score: 0.321351
Param Grid: dict_items([('max_depth', 20), ('max_bins', 6168), ('info_type', 'gini')])
Out[13]:

,F_0.5 Score,Max Depth,Max Bins,Info Type
0,0.305253,5,6168,gini
1,0.324010,10,6168,gini
2,0.321351,20,6168,gini


##Random Forest

In [0]:
Eval_rf = custom_CV(train_test_window, pre_pipe, 'rf', 'down', 5)
Eval_rf

F Score: 0.279684
Param Grid: dict_items([('num_trees', 10), ('depth', 3), ('criterion', 'gini'), ('substrategy', 'sqrt')])
F Score: 0.288759
Param Grid: dict_items([('num_trees', 10), ('depth', 3), ('criterion', 'gini'), ('substrategy', 'all')])
F Score: 0.288417
Param Grid: dict_items([('num_trees', 10), ('depth', 5), ('criterion', 'gini'), ('substrategy', 'sqrt')])
F Score: 0.304559
Param Grid: dict_items([('num_trees', 10), ('depth', 5), ('criterion', 'gini'), ('substrategy', 'all')])
F Score: 0.296859
Param Grid: dict_items([('num_trees', 10), ('depth', 10), ('criterion', 'gini'), ('substrategy', 'sqrt')])
F Score: 0.323125
Param Grid: dict_items([('num_trees', 10), ('depth', 10), ('criterion', 'gini'), ('substrategy', 'all')])
F Score: 0.294743
Param Grid: dict_items([('num_trees', 20), ('depth', 3), ('criterion', 'gini'), ('substrategy', 'sqrt')])

#References

 - https://machinelearningmastery.com/k-fold-cross-validation/
 - https://www.analyticsvidhya.com/blog/2019/11/build-machine-learning-pipelines-pyspark/
 - https://medium.com/@junwan01/oversampling-and-undersampling-with-pyspark-5dbc25cdf253